In [121]:
from os import listdir
from pandas import DataFrame
from itertools import chain
from gensim.utils import tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer
from numpy import isnan
from requests import get, exceptions
from itertools import chain
from nltk import sent_tokenize
import pymorphy2 
from nltk.stem.snowball import SnowballStemmer

In [2]:
DVACH = 'https://2ch.hk/'
BOARD = 'pr'

dvach_page = get(DVACH + BOARD + '/catalog.json').json()
threads = [i['num'] for i in dvach_page['threads']]

In [6]:
documents = []

for every_thread in threads[:50]:
    thread = get(DVACH +  BOARD + '/res/' + every_thread + '.json', timeout=5).json()
    [documents.append(sent_tokenize(cut(i['comment']))) for i in thread['threads'][0]['posts'] if len(cut(i['comment'])) > 2]

documents = list(chain.from_iterable(documents))
len(documents)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


33802

In [167]:
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')
morph = pymorphy2.MorphAnalyzer()

def my_tokenize(sent):
    sents = list(alpha_tokenizer.tokenize(sent))
    return set([morph.parse(sent.lower())[0].normal_form for sent in sents if sent.lower() not in stop and len(sent.lower()) > 1])

model_data = [my_tokenize(sent) for sent in documents]

In [166]:
stop = stopwords.words('russian')
stop += stopwords.words('english')
my_stop = ['весь', 'всякий', 'тот', 'это', 'самое', 'каким', 'сама', 'никак', 'она', 'она', 'оно', 'какие', 'какого', 'которая', 'многое', 'чему', 'всему', 'раза', 'сразу', 'весь', 'раз', 'пор', 'например', 'вроде', 'которые', 'который', 'просто', 'очень', 'почему', 'вообще', 'ещё', 'типа', 'ради', 'всё', 'хотя', 'тебе', 'нужно', 'пока']
my_stop_verb = ['сделать', 'можешь', 'могу', 'могут', 'делать', 'будешь', 'быть', 'будут', 'смочь']
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop]
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop_verb]
dvach_stop = ['такое', 'лол', 'бля']
stop += dvach_stop

In [173]:
min_count = 1
size = 600
window = 40

model = Word2Vec(model_data, min_count=min_count, size=size, window=window,  workers=4)

In [175]:
model.most_similar(morph.parse('галера')[0].normal_form)

[('быть', 0.9999947547912598),
 ('стать', 0.9999947547912598),
 ('сидеть', 0.9999947547912598),
 ('думать', 0.999994695186615),
 ('такой', 0.9999945759773254),
 ('программист', 0.9999945759773254),
 ('человек', 0.9999945163726807),
 ('работа', 0.9999944567680359),
 ('уровень', 0.9999944567680359),
 ('взять', 0.9999944567680359)]

In [67]:
from re import sub, compile

def cut(data):
    r = compile(r'<.*?>|>>\d*|\(OP\)|&#(\d*);|&quot;|&gt;|(http|https):.*')
    return r.sub('', punctuate_word(punctuate_sent((data))))

def punctuate_sent(data):
    r = compile(r'([a-zA-Zа-яА-Я])([.!\?])')
    return r.sub(r'\1. ', data)

def punctuate_word(data):
    r = compile(r'([a-zA-Zа-яА-Я])([,])')
    return r.sub(r'\1, ', data)

def pos_cut(s):
    s = sub(r"ADJF", "ADJ", s)
    s = sub(r"ADVB", "ADV", s)
    s = sub(r"INFN", "VERB", s)
    return s

def topic_cut(word):
    return sub('[^a-zA-Zа-яА-Я]', '', word)

def check_for_rus_lan(word):
    regex = compile('[А-Яа-я]+$')
    if regex.match(word):
        return True
    return False